This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '5vsyuqexs6ryzJfHvRHp'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [48]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-01-01&api_key=' + API_KEY
resp = requests.get(url)
resp_json = resp.json()

In [49]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
resp_json

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-01-24) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-02-07T22:16:21.794Z',
  'newest_available_date': '2020-02-07',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-01-01',
  'end_date': '2017-01-01',
  'data': [],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [53]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY
resp = requests.get(url)
resp.status_code == 200

True

In [55]:
#2. Convert the returned JSON object into a Python dictionary.
resp_json = resp.json()
type(resp_json)

dict

In [82]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.
import operator

# Get all values of opening price and dates
data = resp_json['dataset']['data']

# Index of Opening in the list
idx_open = resp_json['dataset']['column_names'].index('Open')
# Index of Date in the list
idx_date = resp_json['dataset']['column_names'].index('Date')

opening_date_pairs = [(data[idx][idx_date], data[idx][idx_open]) for idx in range(len(data)) if data[idx][idx_open] is not None]
max_opening = max(opening_date_pairs, key = operator.itemgetter(idx_open))
min_opening = min(opening_date_pairs, key = operator.itemgetter(idx_open))
print("The max_opening date was :", max_opening[idx_date],"and value was :",max_opening[idx_open] )
print("The min_opening date was :", min_opening[idx_date],"and value was :",min_opening[idx_open] )

The max_opening date was : 2017-12-14 and value was : 53.11
The min_opening date was : 2017-01-24 and value was : 34.0


In [104]:
#4. What was the largest change in any one day (based on High and Low price)?

# Index of Date in the list
idx_date = resp_json['dataset']['column_names'].index('Date')
# Index of Opening in the list
idx_high = resp_json['dataset']['column_names'].index('High')
# Index of Date in the list
idx_low = resp_json['dataset']['column_names'].index('Low')

largest_change = [(data[idx][idx_date], abs(data[idx][idx_high] - data[idx][idx_low])) for idx in range(len(data))]
max_intraday_change = max(largest_change, key = operator.itemgetter(1))
print("The max_change date was :", max_intraday_change[idx_date],"and change was :",round(max_intraday_change[1],2) )

The max_change date was : 2017-05-11 and change was : 2.81


In [103]:
#5. What was the largest change between any two days (based on Closing Price)?

# Index of Closing Price in the list
idx_close = resp_json['dataset']['column_names'].index('Close')

close_vals = [data[idx][idx_close] for idx in range(len(data)) if data[idx][idx_close] is not None]

print("The largest close price change was :", round(max(list(map(abs,map(operator.sub, close_vals, close_vals[1:])))),2))

The largest close price change was : 2.56


In [102]:
#6. What was the average daily trading volume during this year?

# Index of Closing Price in the list
idx_vol = resp_json['dataset']['column_names'].index('Traded Volume')

traded_vol = [data[idx][idx_vol] for idx in range(len(data)) if data[idx][idx_vol] is not None]

print("The the average daily trading volume in 2017 was :", round(sum(traded_vol) / len(traded_vol),2))

The the average daily trading volume was : 89124.34


In [115]:
# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

def get_median(vol):
    if (len(vol) > 0):
        sorted_vol = sorted(vol, reverse=False)

        vol_length = len(sorted_vol)

        if vol_length %2 == 0:
           return round((sorted_vol[vol_length//2 +1] +  sorted_vol[vol_length//2 -1])/2,2)
        else:
           return sorted_vol[vol_length//2]
    else:
        return 0.0

print("The median trading volume during the year 2017 was", get_median(traded_vol))


The median trading volume during the year 2017 was 76286.0
